In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape} {X.dtype}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28]) torch.float32
Shape of y: torch.Size([64]) torch.int64


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [5]:
def train(dataloader, model, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = F.nll_loss(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss +=  F.nll_loss(pred, y, reduction='sum').item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
device = torch.device("cpu")
torch.set_num_threads(1) 

In [7]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, optimizer)
    test(test_dataloader, model)
    scheduler.step()
print("Done!")

Epoch 1
-------------------------------
loss: 2.332420  [    0/60000]
loss: 0.580163  [ 6400/60000]
loss: 0.429841  [12800/60000]
loss: 0.466516  [19200/60000]
loss: 0.414946  [25600/60000]
loss: 0.421107  [32000/60000]
loss: 0.392035  [38400/60000]
loss: 0.485541  [44800/60000]
loss: 0.345199  [51200/60000]
loss: 0.345817  [57600/60000]
Test Error: 
 Accuracy: 88.5%, Avg loss: 20.201303 

Epoch 2
-------------------------------
loss: 0.168893  [    0/60000]
loss: 0.355841  [ 6400/60000]
loss: 0.162373  [12800/60000]
loss: 0.334505  [19200/60000]
loss: 0.368560  [25600/60000]
loss: 0.338705  [32000/60000]
loss: 0.253481  [38400/60000]
loss: 0.396986  [44800/60000]
loss: 0.346941  [51200/60000]
loss: 0.236526  [57600/60000]
Test Error: 
 Accuracy: 90.6%, Avg loss: 17.669085 

Epoch 3
-------------------------------
loss: 0.136695  [    0/60000]
loss: 0.276821  [ 6400/60000]
loss: 0.166020  [12800/60000]
loss: 0.249754  [19200/60000]
loss: 0.233135  [25600/60000]
loss: 0.351762  [32000/6

In [9]:
input_names = ["image"]
output_names = ["prediction"]
dummy_input = torch.randn(10000, 1, 28, 28)
torch.onnx.export(model,
                  dummy_input,
                  "fmnist.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [10]:
torch.manual_seed(2333)
torch.set_num_interop_threads(1)
dummy_input = torch.rand(10000, 1, 28, 28, dtype=torch.float32, device=device)

In [11]:
%%time

with torch.no_grad():
    y = model(dummy_input)
y.shape

CPU times: user 7.96 s, sys: 3.88 s, total: 11.8 s
Wall time: 11.8 s


torch.Size([10000, 10])

In [12]:
import numpy as np

np_input = dummy_input.detach().numpy()
with open('dummy_64_fminist_in.dat', 'wb') as f:
    np_input.tofile(f)
    
np_output = y.detach().numpy()
with open('dummy_64_fminist_out.dat', 'wb') as f:
    np_output.tofile(f)
